In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("parsed_logs.csv")

In [3]:
df

,start_dt,config,ecc,hostname,it,ker_time,acc_time,ker_err,acc_err,sdc,critical_sdc
0,2022-09-26 05:27:45,c100_res44_test_02_relu6-bn_nanfilter,OFF,carolm20001,259,0.064098,16.797450,84,84,1,0
1,2022-09-26 05:27:45,c100_res44_test_02_relu6-bn_nanfilter,OFF,carolm20001,263,0.064022,17.053545,4096,4180,1,0
2,2022-09-26 05:27:45,c100_res44_test_02_relu6-bn_nanfilter,OFF,carolm20001,278,0.064063,18.014337,4096,8276,1,0
3,2022-09-26 05:27:45,c100_res44_test_02_relu6-bn_nanfilter,OFF,carolm20001,280,0.064022,18.142436,8,8284,1,0
4,2022-09-26 05:27:45,c100_res44_test_02_relu6-bn_nanfilter,OFF,carolm20001,287,0.064076,18.590966,4096,12380,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2816,2022-09-26 09:52:25,c10_res44_test_02_bn-relu6,OFF,carolp20001,94,0.036567,3.541578,5,5,1,0
2817,2022-09-25 16:20:31,c100_res44_test_02_relu6-bn,OFF,carolp20001,1765,0.036693,64.890995,100,100,1,0
2818,2022-09-25 16:20:31,c100_res44_test_02_relu6-bn,OFF,carolp20001,1771,0.047220,65.145673,4096,4196,1,0
2819,2022-09-26 13:06:21,c10_res44_test_02_bn-relu6,OFF,carolp20001,265,0.036785,9.946177,10,10,1,0


In [4]:
df["config"].unique()

array(['c100_res44_test_02_relu6-bn_nanfilter',
       'c100_res44_test_02_relu6-bn', 'c100_res44_test_01_bn-relu_base',
       'c10_res44_test_01_bn-relu_base',
       'c10_res44_test_02_relu6-bn_nanfilter',
       'c10_res44_test_02_relu6-bn', 'cifar100_mobilenetv2_x1_4',
       'c100_res44_test_02_gelu6_nans', 'c10_res44_test_02_bn-relu6'],
      dtype=object)

In [5]:
configs = dict(
    # Baseline C100
    BaseC100="c100_res44_test_01_bn-relu_base.yaml",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="c100_res44_test_02_relu6-bn.yaml",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    # Order inversion + nan filter + Relu6
    OINanRelu6C100="c100_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C100
    GeluNanC100="c100_res44_test_02_gelu6_nans.yaml",

    # Baseline C10
    BaseC10="c10_res44_test_01_bn-relu_base.yaml",
    # "c10_res44_test_02_bn-relu6.yaml",
    BnRelu6="c10_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C10
    OrderInvC10="c10_res44_test_02_relu6-bn.yaml",
    # "c10_res44_test_02_bn-relu6_base.yaml",
    OINanRelu6C10="c10_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C10
    GeluNanC10="c10_res44_test_02_gelu6_nans.yaml",

    MobileNetC100="cifar100_mobilenetv2_x1_4.yaml",
    MobileNetC10="cifar10_mobilenetv2_x1_4.yaml"
)
configs_inv = {v.replace(".yaml", ""): k for k, v in configs.items()}

In [6]:
configs_df = pd.DataFrame([configs]).T.reset_index().rename(columns={0:"YAML", "index": "config"})

In [7]:
configs_df

,config,YAML
0,BaseC100,c100_res44_test_01_bn-relu_base.yaml
1,OrderInvC100,c100_res44_test_02_relu6-bn.yaml
2,OINanRelu6C100,c100_res44_test_02_relu6-bn_nanfilter.yaml
3,GeluNanC100,c100_res44_test_02_gelu6_nans.yaml
4,BaseC10,c10_res44_test_01_bn-relu_base.yaml
5,BnRelu6,c10_res44_test_02_bn-relu6.yaml
6,OrderInvC10,c10_res44_test_02_relu6-bn.yaml
7,OINanRelu6C10,c10_res44_test_02_relu6-bn_nanfilter.yaml
8,GeluNanC10,c10_res44_test_02_gelu6_nans.yaml
9,MobileNetC100,cifar100_mobilenetv2_x1_4.yaml


In [8]:
df["config"] = df["config"].apply(lambda x: configs_inv[x])

In [9]:
df["count"] = 1

In [10]:
grouped_df = df.groupby(["hostname", "config"]).sum()

In [11]:
grouped_df

it   ker_time      acc_time  ker_err  \
hostname    config                                                      
carolm20001 BaseC100          54502  13.383705   2379.867973   420120   
            OINanRelu6C100    63855  17.464981   4160.807624   407952   
            OrderInvC100      44598  12.336246   2905.855822   278677   
carolp20001 BaseC100        2006129  10.827075  37930.895263   337859   
            BnRelu6          252702   3.125701   9273.957378     5084   
            GeluNanC100      454051   8.677411  16667.342402   209870   
            MobileNetC100     76565  13.434314   7162.338040    23312   
            OINanRelu6C100   830979  13.352289  30574.568313   263263   
            OrderInvC100     746910  12.192618  27524.076486   240627   
carolp20002 BaseC10         1543923   7.346595  36180.194687    28891   
            OINanRelu6C10   1068213  15.272654  44740.957634    23097   
            OrderInvC10      211718   2.906523   8803.612010     8130   

                            acc_err  sdc  critical_sdc  count  
hostname    config                                             
carolm20001 BaseC100        1738607  249            14    249  
            OINanRelu6C100  1528901  242             7    242  
            OrderInvC100     876376  175             8    175  
carolp20001 BaseC100         888767  447            15    447  
            BnRelu6           14577   78             0     78  
            GeluNanC100      414405  215             5    215  
            MobileNetC100     61665  115             2    115  
            OINanRelu6C100   460554  298             6    298  
            OrderInvC100     439031  300             4    300  
carolp20002 BaseC10           82885  280             4    280  
            OINanRelu6C10     36377  354             3    354  
            OrderInvC10       20143   68             2     68

In [12]:
grouped_df[["sdc", "critical_sdc", "ker_err", "count"]].to_csv("avf.csv")

In [13]:
grouped_df[["sdc", "critical_sdc", "acc_time"]]

sdc  critical_sdc      acc_time
hostname    config                                         
carolm20001 BaseC100        249            14   2379.867973
            OINanRelu6C100  242             7   4160.807624
            OrderInvC100    175             8   2905.855822
carolp20001 BaseC100        447            15  37930.895263
            BnRelu6          78             0   9273.957378
            GeluNanC100     215             5  16667.342402
            MobileNetC100   115             2   7162.338040
            OINanRelu6C100  298             6  30574.568313
            OrderInvC100    300             4  27524.076486
carolp20002 BaseC10         280             4  36180.194687
            OINanRelu6C10   354             3  44740.957634
            OrderInvC10      68             2   8803.612010

In [14]:
grouped_df["sdc_rate"] = grouped_df["sdc"] / grouped_df["acc_time"]
grouped_df["critical_sdc_rate"] = grouped_df["critical_sdc"] / grouped_df["acc_time"]

In [15]:
grouped_df

it   ker_time      acc_time  ker_err  \
hostname    config                                                      
carolm20001 BaseC100          54502  13.383705   2379.867973   420120   
            OINanRelu6C100    63855  17.464981   4160.807624   407952   
            OrderInvC100      44598  12.336246   2905.855822   278677   
carolp20001 BaseC100        2006129  10.827075  37930.895263   337859   
            BnRelu6          252702   3.125701   9273.957378     5084   
            GeluNanC100      454051   8.677411  16667.342402   209870   
            MobileNetC100     76565  13.434314   7162.338040    23312   
            OINanRelu6C100   830979  13.352289  30574.568313   263263   
            OrderInvC100     746910  12.192618  27524.076486   240627   
carolp20002 BaseC10         1543923   7.346595  36180.194687    28891   
            OINanRelu6C10   1068213  15.272654  44740.957634    23097   
            OrderInvC10      211718   2.906523   8803.612010     8130   

                            acc_err  sdc  critical_sdc  count  sdc_rate  \
hostname    config                                                        
carolm20001 BaseC100        1738607  249            14    249  0.104628   
            OINanRelu6C100  1528901  242             7    242  0.058162   
            OrderInvC100     876376  175             8    175  0.060223   
carolp20001 BaseC100         888767  447            15    447  0.011785   
            BnRelu6           14577   78             0     78  0.008411   
            GeluNanC100      414405  215             5    215  0.012899   
            MobileNetC100     61665  115             2    115  0.016056   
            OINanRelu6C100   460554  298             6    298  0.009747   
            OrderInvC100     439031  300             4    300  0.010900   
carolp20002 BaseC10           82885  280             4    280  0.007739   
            OINanRelu6C10     36377  354             3    354  0.007912   
            OrderInvC10       20143   68             2     68  0.007724   

                            critical_sdc_rate  
hostname    config                             
carolm20001 BaseC100                 0.005883  
            OINanRelu6C100           0.001682  
            OrderInvC100             0.002753  
carolp20001 BaseC100                 0.000395  
            BnRelu6                  0.000000  
            GeluNanC100              0.000300  
            MobileNetC100            0.000279  
            OINanRelu6C100           0.000196  
            OrderInvC100             0.000145  
carolp20002 BaseC10                  0.000111  
            OINanRelu6C10            0.000067  
            OrderInvC10              0.000227

In [16]:
grouped_df[["sdc", "critical_sdc", "acc_time", "sdc_rate", "critical_sdc_rate"]]

sdc  critical_sdc      acc_time  sdc_rate  \
hostname    config                                                      
carolm20001 BaseC100        249            14   2379.867973  0.104628   
            OINanRelu6C100  242             7   4160.807624  0.058162   
            OrderInvC100    175             8   2905.855822  0.060223   
carolp20001 BaseC100        447            15  37930.895263  0.011785   
            BnRelu6          78             0   9273.957378  0.008411   
            GeluNanC100     215             5  16667.342402  0.012899   
            MobileNetC100   115             2   7162.338040  0.016056   
            OINanRelu6C100  298             6  30574.568313  0.009747   
            OrderInvC100    300             4  27524.076486  0.010900   
carolp20002 BaseC10         280             4  36180.194687  0.007739   
            OINanRelu6C10   354             3  44740.957634  0.007912   
            OrderInvC10      68             2   8803.612010  0.007724   

                            critical_sdc_rate  
hostname    config                             
carolm20001 BaseC100                 0.005883  
            OINanRelu6C100           0.001682  
            OrderInvC100             0.002753  
carolp20001 BaseC100                 0.000395  
            BnRelu6                  0.000000  
            GeluNanC100              0.000300  
            MobileNetC100            0.000279  
            OINanRelu6C100           0.000196  
            OrderInvC100             0.000145  
carolp20002 BaseC10                  0.000111  
            OINanRelu6C10            0.000067  
            OrderInvC10              0.000227